In [113]:
import pandas as pd
import numpy as np

In [114]:
HOBO_low = pd.read_csv("HOBO\Group3-Low.csv", encoding="windows-1252")
HOBO_mid = pd.read_csv("HOBO\Group3-Mid.csv", encoding="windows-1252")
HOBO_top = pd.read_csv("HOBO\Group3-Top.csv", encoding="windows-1252")
CUBE = CUBE = pd.read_excel("CUBE\Group 3-Cube.xlsx")
results_1A = pd.read_csv("results_TOP.csv")

In [115]:
v_a_low = HOBO_low.iloc[:, 3]
v_a_mid = HOBO_mid.iloc[:, 3]
v_a_top = HOBO_top.iloc[:, 3]
t_mean_op = results_1A.iloc[:,6]

In [116]:
def draught_rate_comfort(v_ave, t_op, interval):

    DR_standards = np.zeros((len(v_ave), 3))
    cols = ["Draught Rate", "ASHRAE Standard", "ISO 7730 Standard"]
    DR_standards = pd.DataFrame(DR_standards, columns=cols)

    for i in range(0, len(v_ave-interval), interval):
        v_sum_squared = 0
        v_rms = 0
        for j in range(interval):
            if i+j == len(v_ave-interval)-1:
                v = max(v_ave[i+j], 0.05)
                v_sum_squared += v*v
                v_rms = ((1/(j+1))*v_sum_squared)**0.5
                break
            v = max(v_ave[i+j], 0.05)
            v_sum_squared += v*v
        v_rms = ((1/interval)*v_sum_squared)**0.5
    
        for j in range(interval):
            if i+j == len(v_ave-interval)-1:
                v = max(v_ave[i+j], 0.05)
                TU = v_rms/v_ave[i+j]
                DR_standards.iloc[i+j,0] = (34 - t_op[i+j])*((v-0.05)**0.62)*(0.37*v*TU+3.14) 
                break
            v = max(v_ave[i+j], 0.05)
            TU = v_rms/v_ave[i+j]
            DR_standards.iloc[i+j,0] = (34 - t_op[i+j])*((v-0.05)**0.62)*(0.37*v*TU+3.14)

    for k in range(len(v_ave)):
        if DR_standards.iloc[k,0] <= 10:
            DR_standards.iloc[k,2] = "Cat I"
        elif DR_standards.iloc[k,0] <= 20:
            DR_standards.iloc[k,2] = "Cat II"
        elif DR_standards.iloc[k,0] <= 30:
            DR_standards.iloc[k,2] = "Cat III"
        else:
            DR_standards.iloc[k,2] = "X"

    for k in range(len(v_ave)):
        if DR_standards.iloc[k,0] <= 20:
            DR_standards.iloc[k,1] = "ok"
        else:
            DR_standards.iloc[k,1] = "X"

    return DR_standards

In [122]:
dr_results = draught_rate_comfort(v_a_mid, t_mean_op, 60)
print(dr_results[dr_results["ISO 7730 Standard"] == "Cat II"])
print(dr_results[dr_results["ISO 7730 Standard"] == "Cat III"])
print(dr_results[dr_results["ISO 7730 Standard"] == "X"])
print(dr_results[dr_results["ASHRAE Standard"] == "X"])



C:\Users\aarne\AppData\Local\Temp\ipykernel_23056\1337488408.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Cat I' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  DR_standards.iloc[k,2] = "Cat I"
C:\Users\aarne\AppData\Local\Temp\ipykernel_23056\1337488408.py:42: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'ok' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  DR_standards.iloc[k,1] = "ok"


      Draught Rate ASHRAE Standard ISO 7730 Standard
1009     10.167783              ok            Cat II
1163     10.724004              ok            Cat II
1172     17.622978              ok            Cat II
1173     15.422622              ok            Cat II
1174     10.962723              ok            Cat II
...            ...             ...               ...
7019     12.080638              ok            Cat II
7021     10.201857              ok            Cat II
7034     10.095705              ok            Cat II
7078     10.252691              ok            Cat II
7180     10.339975              ok            Cat II

[723 rows x 3 columns]
      Draught Rate ASHRAE Standard ISO 7730 Standard
1189     27.288364               X           Cat III
1191     29.405743               X           Cat III
1192     27.949101               X           Cat III
1193     28.850638               X           Cat III
1194     23.954597               X           Cat III
...            ...    

In [118]:
Tpr_north = CUBE.iloc[:,4]
Tpr_south = CUBE.iloc[:,6]
Tpr_east = CUBE.iloc[:,5]
Tpr_west = CUBE.iloc[:,7]
Tpr_up = CUBE.iloc[:,3]
Tpr_down = CUBE.iloc[:,8]

In [ ]:
def radiant_temperature_asymmetry_comfort(n, s, e, w, u, d):
    DT_ns = n - s
    DT_ew = e - w
    DT_ud = u - d
    
    ISO_7730_warm_ns = []
    ISO_7730_cold_ns = []
    ASHRAE_warm_ns = []
    ASHRAE_cold_ns = []

    ISO_7730_warm_ew = []
    ISO_7730_cold_ew = []
    ASHRAE_warm_ew = []
    ASHRAE_cold_ew = []

    ISO_7730_warm_ud = []
    ISO_7730_cold_ud = []
    ASHRAE_warm_ud = []
    ASHRAE_cold_ud = []

    for i in range(len(DT_ns)):
        if DT_ns[i] > -10:
            ISO_7730_cold_ns.append("Cat I, II")
            ASHRAE_cold_ns.append("OK")
        elif DT_ns[i] > -13:
            ISO_7730_cold_ns.append("Cat III")
        else:
            ISO_7730_cold_ns.append("X")
            ASHRAE_cold_ns.append("X")

        if DT_ns[i] < 23:
            ISO_7730_warm_ns.append("Cat I, II")
            ASHRAE_warm_ns.append("OK")
        elif DT_ns[i] < 35:
            ISO_7730_warm_ns.append("Cat III")
        else:
            ISO_7730_warm_ns.append("X")
            ASHRAE_warm_ns.append("X")

    # Looping through each element in DT_ew
    for i in range(len(DT_ew)):  
        if DT_ew[i] > -10:
            ISO_7730_cold_ew.append("Cat I, II")
            ASHRAE_cold_ew.append("OK")
        elif DT_ew[i] > -13:
            ISO_7730_cold_ew.append("Cat III")
        else:
            ISO_7730_cold_ew.append("X")
            ASHRAE_cold_ew.append("X")

        if DT_ew[i] < 23:
            ISO_7730_warm_ew.append("Cat I, II")
            ASHRAE_warm_ew.append("OK")
        elif DT_ew[i] < 35:
            ISO_7730_warm_ew.append("Cat III")
        else:
            ISO_7730_warm_ew.append("X")
            ASHRAE_warm_ew.append("X")

    # Looping through each element in DT_ud
    for i in range(len(DT_ud)):  
        if DT_ud[i] > -14:
            ISO_7730_cold_ud.append("Cat I, II")
            ASHRAE_cold_ud.append("OK")
        elif DT_ud[i] > -18:
            ISO_7730_cold_ud.append("Cat III")
        else:
            ISO_7730_cold_ud.append("X")
            ASHRAE_cold_ud.append("X")

        if DT_ud[i] < 5:
            ISO_7730_warm_ud.append("Cat I, II")
            ASHRAE_warm_ud.append("OK")
        elif DT_ud[i] < 7:
            ISO_7730_warm_ud.append("Cat III")
        else:
            ISO_7730_warm_ud.append("X")
            ASHRAE_warm_ud.append("X")

    result = np.column_stack([DT_ns, ISO_7730_warm_ns, ISO_7730_cold_ns, ASHRAE_warm_ns, ASHRAE_cold_ns,
              DT_ew, ISO_7730_warm_ew, ISO_7730_cold_ew, ASHRAE_warm_ew, ASHRAE_cold_ew,
              DT_ud, ISO_7730_warm_ud, ISO_7730_cold_ud, ASHRAE_warm_ud, ASHRAE_cold_ud])
    
    cols = [
    "DT_ns", "ISO_7730_warm_ns", "ISO_7730_cold_ns", "ASHRAE_warm_ns", "ASHRAE_cold_ns",
    "DT_ew", "ISO_7730_warm_ew", "ISO_7730_cold_ew", "ASHRAE_warm_ns", "ASHRAE_cold_ew",
    "DT_ud", "ISO_7730_warm_ud", "ISO_7730_cold_ud", "ASHRAE_warm_ud", "ASHRAE_cold_ud"
    ]

    df = pd.DataFrame(result, columns=cols)
    
    return df



In [124]:
results_asymmetry = radiant_temperature_asymmetry_comfort(Tpr_north, Tpr_south, Tpr_east, Tpr_west, Tpr_up, Tpr_down)
print(results_asymmetry[(results_asymmetry["ISO_7730_warm_ns"] == "Cat III") | (results_asymmetry["ISO_7730_cold_ns"] == "Cat III")])
print(results_asymmetry[(results_asymmetry["ISO_7730_warm_ew"] == "Cat III") | (results_asymmetry["ISO_7730_cold_ew"] == "Cat III")])
print(results_asymmetry[(results_asymmetry["ISO_7730_warm_ud"] == "Cat III") | (results_asymmetry["ISO_7730_cold_ud"] == "Cat III")])
print(results_asymmetry[(results_asymmetry["ISO_7730_warm_ns"] == "X") | (results_asymmetry["ISO_7730_cold_ns"] == "X")])
print(results_asymmetry[(results_asymmetry["ISO_7730_warm_ew"] == "X") | (results_asymmetry["ISO_7730_cold_ew"] == "X")])
print(results_asymmetry[(results_asymmetry["ISO_7730_warm_ud"] == "X") | (results_asymmetry["ISO_7730_cold_ud"] == "X")])
print(results_asymmetry[(results_asymmetry["ASHRAE_warm_ns"] == "X") | (results_asymmetry["ASHRAE_cold_ns"] == "X")])
print(results_asymmetry[(results_asymmetry["ASHRAE_warm_ew"] == "X") | (results_asymmetry["ASHRAE_cold_ew"] == "X")])
print(results_asymmetry[(results_asymmetry["ASHRAE_warm_ud"] == "X") | (results_asymmetry["ASHRAE_cold_ud"] == "X")])

Empty DataFrame
Columns: [DT_ns, ISO_7730_warm_ns, ISO_7730_cold_ns, ASHRAE_warm_ns, ASHRAE_cold_ns, DT_ew, ISO_7730_warm_ew, ISO_7730_cold_ew, ASHRAE_warm_ew, ASHRAE_cold_ew, DT_ud, ISO_7730_warm_ud, ISO_7730_cold_ud, ASHRAE_warm_ud, ASHRAE_cold_ud]
Index: []
Empty DataFrame
Columns: [DT_ns, ISO_7730_warm_ns, ISO_7730_cold_ns, ASHRAE_warm_ns, ASHRAE_cold_ns, DT_ew, ISO_7730_warm_ew, ISO_7730_cold_ew, ASHRAE_warm_ew, ASHRAE_cold_ew, DT_ud, ISO_7730_warm_ud, ISO_7730_cold_ud, ASHRAE_warm_ud, ASHRAE_cold_ud]
Index: []
Empty DataFrame
Columns: [DT_ns, ISO_7730_warm_ns, ISO_7730_cold_ns, ASHRAE_warm_ns, ASHRAE_cold_ns, DT_ew, ISO_7730_warm_ew, ISO_7730_cold_ew, ASHRAE_warm_ew, ASHRAE_cold_ew, DT_ud, ISO_7730_warm_ud, ISO_7730_cold_ud, ASHRAE_warm_ud, ASHRAE_cold_ud]
Index: []
Empty DataFrame
Columns: [DT_ns, ISO_7730_warm_ns, ISO_7730_cold_ns, ASHRAE_warm_ns, ASHRAE_cold_ns, DT_ew, ISO_7730_warm_ew, ISO_7730_cold_ew, ASHRAE_warm_ew, ASHRAE_cold_ew, DT_ud, ISO_7730_warm_ud, ISO_7730_cold_ud